In [7]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [8]:
print(tf.__version__)

2.1.0


In [9]:
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

Shuffling and writing examples to C:\Users\raada\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incomplete29NPDS\imdb_reviews-train.tfrecord


Shuffling and writing examples to C:\Users\raada\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incomplete29NPDS\imdb_reviews-test.tfrecord


Shuffling and writing examples to C:\Users\raada\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incomplete29NPDS\imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to C:\Users\raada\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


In [10]:
train_data, test_data = imdb["train"], imdb["test"]

In [12]:
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()

for sentence, label in train_data:
    training_sentences.append(str(sentence.numpy()))
    training_labels.append(label.numpy())
    
for sentence, label in test_data:
    testing_sentences.append(str(sentence.numpy()))
    testing_labels.append(label.numpy())

In [13]:
import numpy as np

In [14]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [17]:
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [18]:
vocab_size = 10000
embedding_dim =  16
max_length = 120
trunc_type = "post"
oov_token = "<OOV>"

In [19]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)

In [20]:
padded = pad_sequences(sequences, maxlen = max_length, truncating = trunc_type)

In [21]:
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padding = pad_sequences(testing_sequences, maxlen = max_length, truncating = trunc_type)

In [48]:
reverse_word_index = dict([(value,key) for (key, value) in word_index.items()])

def decode_review(text):
    return " ".join([reverse_word_index.get(i, "?") for i in text])

print(decode_review(padded[1]))
print()
print(training_sentences[1])

? ? ? ? ? ? ? b'i have been known to fall asleep during films but this is usually due to a combination of things including really tired being warm and comfortable on the <OOV> and having just eaten a lot however on this occasion i fell asleep because the film was rubbish the plot development was constant constantly slow and boring things seemed to happen but with no explanation of what was causing them or why i admit i may have missed part of the film but i watched the majority of it and everything just seemed to happen of its own <OOV> without any real concern for anything else i cant recommend this film at all '

b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was constant. Constantly slow and boring. Things seemed to happen, but with no explanation of

In [51]:
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
    keras.layers.Flatten(),
    keras.layers.Dense(6, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])

In [52]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [53]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_3 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [54]:
epochs = 10
model.fit(padded,training_labels_final,epochs=epochs,validation_data=(testing_padding,testing_labels_final))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 3s 113us/sample - loss: 0.4945 - accuracy: 0.7412 - val_loss: 0.3826 - val_accuracy: 0.8268
Epoch 2/10
25000/25000 [==============================] - 2s 93us/sample - loss: 0.2439 - accuracy: 0.9062 - val_loss: 0.4156 - val_accuracy: 0.8198
Epoch 3/10
25000/25000 [==============================] - 2s 96us/sample - loss: 0.1002 - accuracy: 0.9739 - val_loss: 0.5151 - val_accuracy: 0.8066
Epoch 4/10
25000/25000 [==============================] - 2s 98us/sample - loss: 0.0285 - accuracy: 0.9955 - val_loss: 0.5974 - val_accuracy: 0.8042
Epoch 5/10
25000/25000 [==============================] - 3s 101us/sample - loss: 0.0068 - accuracy: 0.9995 - val_loss: 0.6842 - val_accuracy: 0.8056
Epoch 6/10
25000/25000 [==============================] - 3s 101us/sample - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.7431 - val_accuracy: 0.8064
Epoch 7/10
25000/25000 [=============================

In [55]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)         # shape : (vocab_size, embedding_dim)

(10000, 16)


In [57]:
import io

In [58]:
out_v = io.open("vecs.tsv","w",encoding="utf-8")
out_m = io.open("meta.csv","w",encoding="utf-8")
for word_num in range(1, vocab_size):
    word = reverse_word_index[word_num]
    embeddings = weights[word_num]
    out_m.write(word + "\n")
    out_v.write("\t".join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()